In [ ]:
from oura_analysis.loader import OuraDataNumeric
import plotly.express as px
import pandas as pd
import numpy as np

import plotly.graph_objects as go

oura_data = OuraDataNumeric.from_path("../data/oura_2019-06-01_2024-01-01_trends.csv")

hrv_data = oura_data.data_table[["date", "Average HRV"]]
hrv_data.set_index("date", inplace=True)
hrv_data.sort_index(inplace=True)

Lets first create a plot of all the average HRV data.


In [ ]:
px.scatter(hrv_data, y="Average HRV", title="Average HRV over time - all days")

Then lets create rolling averages with different window sizes


In [ ]:
closed_setting = "both"
min_periods_coefficient = 0.9

hrv_data_7_days = hrv_data.rolling(7, closed=closed_setting, min_periods=int(7 * min_periods_coefficient)).agg(["mean", "std"])
hrv_data_7_days.columns = ["Average HRV (7 days): mean", "Average HRV (7 days): std"]
hrv_data_7_days["Average HRV (7 days): mean + std"] = hrv_data_7_days["Average HRV (7 days): mean"] + hrv_data_7_days["Average HRV (7 days): std"]
hrv_data_7_days["Average HRV (7 days): mean - std"] = hrv_data_7_days["Average HRV (7 days): mean"] - hrv_data_7_days["Average HRV (7 days): std"]

hrv_data_28_days = hrv_data.rolling(28, closed=closed_setting, min_periods=int(28 * min_periods_coefficient)).agg(["mean", "std"])
hrv_data_28_days.columns = ["Average HRV (28 days): mean", "Average HRV (28 days): std"]
hrv_data_28_days["Average HRV (28 days): mean + std"] = hrv_data_28_days["Average HRV (28 days): mean"] + hrv_data_28_days["Average HRV (28 days): std"]
hrv_data_28_days["Average HRV (28 days): mean - std"] = hrv_data_28_days["Average HRV (28 days): mean"] - hrv_data_28_days["Average HRV (28 days): std"]

hrv_data_90_days = hrv_data.rolling(90, closed=closed_setting, min_periods=int(90 * min_periods_coefficient)).agg(["mean", "std"])
hrv_data_90_days.columns = ["Average HRV (90 days): mean", "Average HRV (90 days): std"]
hrv_data_90_days["Average HRV (90 days): mean + std"] = hrv_data_90_days["Average HRV (90 days): mean"] + hrv_data_90_days["Average HRV (90 days): std"]
hrv_data_90_days["Average HRV (90 days): mean - std"] = hrv_data_90_days["Average HRV (90 days): mean"] - hrv_data_90_days["Average HRV (90 days): std"]

hrv_data_365_days = hrv_data.rolling(365, closed=closed_setting, min_periods=int(365 * min_periods_coefficient)).agg(["mean", "std"])
hrv_data_365_days.columns = [
    "Average HRV (365 days): mean",
    "Average HRV (365 days): std",
]
hrv_data_365_days["Average HRV (365 days): mean + std"] = hrv_data_365_days["Average HRV (365 days): mean"] + hrv_data_365_days["Average HRV (365 days): std"]
hrv_data_365_days["Average HRV (365 days): mean - std"] = hrv_data_365_days["Average HRV (365 days): mean"] - hrv_data_365_days["Average HRV (365 days): std"]

In [ ]:
# plot all the rolling averages
rolling_plot_data = pd.concat(
    [hrv_data, hrv_data_7_days, hrv_data_28_days, hrv_data_90_days, hrv_data_365_days],
    axis=1,
)
px.scatter(rolling_plot_data, title="Average HRV over time - rolling averages")

In [ ]:
def create_plot_trendline_plot(input_data, subtitle: str = ""):
    # Create a Plotly trace for the mean line
    x_values = input_data["date"]
    mean_values = input_data["rolling_mean"]
    mean_trace = go.Scatter(x=x_values, y=mean_values, mode="lines", name="Mean")

    daily_values = input_data["daily"]
    daily_trace = go.Scatter(x=x_values, y=daily_values, mode="markers", name="Daily")

    upper_bound = input_data["upper_bound"]
    lower_bound = input_data["lower_bound"]
    shaded_area_trace = go.Scatter(
        x=np.concatenate((x_values, x_values[::-1])),
        y=np.concatenate((upper_bound, lower_bound[::-1])),
        fill="tozerox",
        fillcolor="rgba(0,100,80,0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        name="Mean ± Std",
    )

    # Create the layout for the plot
    layout = go.Layout(
        title=f"Mean and Standard Deviation Plot: {subtitle}",
        xaxis=dict(title="X-Axis"),
        yaxis=dict(title="Y-Axis"),
        showlegend=True,
    )

    # Create the figure and add the traces
    fig = go.Figure(data=[daily_trace, mean_trace, shaded_area_trace], layout=layout)

    # Display the plot
    fig.show()

In [ ]:
# plot selected rolling average as shaded trendline
rolling_avg_days = 90
input_data: pd.DataFrame = hrv_data_90_days.loc[
    :,
    (
        f"Average HRV ({rolling_avg_days} days): mean",
        f"Average HRV ({rolling_avg_days} days): mean + std",
        f"Average HRV ({rolling_avg_days} days): mean - std",
    ),
]
input_data["daily"] = hrv_data["Average HRV"]
input_data = input_data.sort_index()
input_data = input_data.reset_index()
input_data = input_data.rename(
    columns={
        f"Average HRV ({rolling_avg_days} days): mean": "rolling_mean",
        f"Average HRV ({rolling_avg_days} days): mean + std": "upper_bound",
        f"Average HRV ({rolling_avg_days} days): mean - std": "lower_bound",
    }
)
subtitle = f"HRV norm - (rolling over {rolling_avg_days} days)"

create_plot_trendline_plot(input_data=input_data, subtitle=subtitle)

In [ ]:
# TODO idea color code points below upper bound